In [62]:
import os
import numpy as np
from tqdm import tqdm_notebook as tqdm
from glob import glob
import cv2

DATA_PATH = r'D:\code\hw\miptcv\code\hw#5\Train'
classes = map(int, os.listdir(DATA_PATH))

## Загрузка данные

In [82]:
## Загрузка данных
def collect_data(data_path, augment=True):
    data = [[] for i in classes]
    for cl in classes:
        for filename in tqdm(glob(os.path.join(data_path, str(cl), '*')), desc='Read {}'.format(cl)):
            data[cl].append(cv2.imread(filename, 0))
    return data

## Выделение признаков

In [195]:
def white_rate(src):
    return float((src > 0).sum()) / src.size

def gradient_x(src):
    return float((src[:, 1:] - src[:, :-1]).sum()) / (src[1:, :].size)

def gradient_y(src):
    return float((src[1:, :] - src[:-1, :]).sum()) / (src[1:, :].size)

def vertical_projection(src):
    return float(sum(src.sum(axis=1) < 10)) / (1 + src.shape[0])

def horizontal_projection(src):
    return float(sum(src.sum(axis=0) < 10)) / (1 + src.shape[1])

def get_fv(src):
    return np.array([white_rate(src),
                      gradient_x(src),
                      gradient_y(src),
                      vertical_projection(src)+horizontal_projection(src)
                    ])
    
def extract_features(src, size=None, aug_codes=[]):
    process_dict = {1: np.fliplr, 2: np.rot90, 3: np.flipud}
    features = list()
    features.append(get_fv(src))
    for code in aug_codes:
        dst = process_dict[code](src)
        features.append(get_fv(dst))
    return np.vstack(features)

Использованные признаки:

1) Доля белых пикселей на изображении, поскольку эта величина по моим наблюдениям различается для различных классов и близка для изображений одного класса

2) Сумма горизонтальных и вертикальных градиентов, что должно помочь отличить привычные штрих-коды от QR-кодов и сканов

3) Отношение длин белых участков на горизонтальных и вертикальных проекциях изображения, что может позволить различать классы 1,2 или 0,4

## Попробуем SVM

SVM считается хорошей моделью для классификации изображений. Используем стандартное ядро RBF для учёта нелинейности.

In [84]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [85]:
raw_data = collect_data(DATA_PATH)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [179]:
aug_codes = []
X = np.vstack([np.vstack([extract_features(src, aug_codes=aug_codes) for src in raw_data[l]]) for l in labels])
y = np.concatenate([[l] * (1+len(aug_codes))*len(raw_data[l]) for l in labels])

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size = 0.8, test_size = 0.2)

In [180]:
clf0 = SVC(random_state=2)
clf0.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=2, shrinking=True,
  tol=0.001, verbose=False)

In [181]:
print'Train set: {}'.format( clf0.score(X_train, y_train))
print 'Val set: {}'.format( clf0.score(X_val, y_val))
print 'Full set: {}'.format( clf0.score(X, y))

Train set: 0.999642857143
Val set: 1.0
Full set: 0.999714285714


## Попробуем искусственно расширить доступную нам выборку
В изображениях тестовой выборки присутствуют изображения, похожие на изображения из обучающей выборки, но повёрнутые на кратные 90 градусам углы. Пополним обучающую выборку подобными изображениями.

In [198]:
aug_codes = [1,2,3]
aX = np.concatenate([np.vstack([extract_features(src, aug_codes=aug_codes) for src in raw_data[l]]) for l in labels])
ay = np.concatenate([[l] * (1+len(aug_codes))*len(raw_data[l]) for l in labels])

In [199]:
aX_train, aX_val, ay_train, ay_val = train_test_split(aX, ay, train_size = 0.8, test_size = 0.2)

In [200]:
clf_a = SVC(random_state=2)
clf_a.fit(aX, ay)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=2, shrinking=True,
  tol=0.001, verbose=False)

In [201]:
print 'Train set: {}'.format( clf_a.score(X_train, y_train))
print 'Val set: {}'.format( clf_a.score(X_val, y_val))
print 'Full set: {}'.format( clf_a.score(X, y))

Train set: 0.999821428571
Val set: 1.0
Full set: 0.999857142857


## Обучим на всей выборке

In [202]:
clf_res = SVC(random_state=2)
clf_res.fit(aX, ay)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=2, shrinking=True,
  tol=0.001, verbose=False)

In [203]:
print 'Full set: {}'.format( clf_a.score(X, y))

Full set: 0.999857142857


In [204]:
def generate_csv_output(model, data_path, output_path):
    raw_data = list()
    file_list = glob(os.path.join(data_path, '*'))
    for filename in tqdm(file_list):
            raw_data.append(cv2.imread(filename, 0))
    features = np.vstack([extract_features(src) for src in raw_data])
    answers = model.predict(features)
    with open(output_path, 'w') as f:
        f.write('# fname,class\n')
        for i in range(len(file_list)):
            f.write('{},{}\n'.format(os.path.basename(file_list[i]),answers[i]))
    

In [206]:
TEST_DATA_PATH = 'Test'
CSV_PATH = r'res.csv'

generate_csv_output(clf_res, 'test', CSV_PATH)

A Jupyter Widget